In [12]:
import pandas as pd
from tensorflow import keras
import tensorflow as tf
import random
from sklearn.utils import shuffle
import scipy.stats
from zipfile import ZipFile 
import optuna
from sklearn.model_selection import train_test_split
from tabulate import tabulate

In [13]:
import numpy as np
import time
from tensorflow.keras import layers, regularizers
from tensorflow.keras.utils import to_categorical

In [14]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

In [15]:
def load_psg_data():
    input_dir = 'PSG_Dataset/'
    x_train = np.load(input_dir +'x_train.npy')
    x_validation = np.load(input_dir +'x_valid.npy')
    x_test = np.load(input_dir +'x_test.npy')
    y_train = np.load(input_dir +'y_train.npy')
    y_validation = np.load(input_dir +'y_valid.npy')
    y_test = np.load(input_dir +'y_test.npy')
    
    return x_train, y_train, x_validation, y_validation, x_test, y_test

In [20]:
def cnn_model_1(x_train, y_train):
    n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
    model = keras.Sequential(
        [
            keras.Input(shape=(n_timesteps,n_features)),
            layers.Conv1D(filters=100, kernel_size=50, activation='relu'),
            layers.Conv1D(filters=100, kernel_size=50, activation='relu'),
            layers.Dropout(0.1),
            layers.MaxPooling1D(pool_size=2),
            layers.Flatten(),
            layers.Dense(100, activation='relu'),
            layers.Dense(n_outputs, activation='softmax')
        ]
    )
    model.summary()
    return model

In [21]:
print("Get PSG-Audio (npy data)")
x_train, y_train, x_valid, y_valid, x_test, y_test \
                         = load_psg_data()
headers = ("Array","shape", "data type")
mydata = [("x_train:", x_train.shape, x_train.dtype),
        ("y_train:", y_train.shape, y_train.dtype),
        ("x_valid:", x_valid.shape, x_valid.dtype),
        ("y_valid:", y_valid.shape, y_valid.dtype),
        ("x_test:", x_test.shape, x_test.dtype),
        ("y_test:", y_test.shape, y_test.dtype)]
print("\n",tabulate(mydata, headers=headers))
print ('\n','-'*72) # just a dashed line

Get PSG-Audio (npy data)

 Array     shape             data type
--------  ----------------  -----------
x_train:  (41469, 500, 12)  float32
y_train:  (41469, 2)        uint8
x_valid:  (15854, 500, 12)  float32
y_valid:  (15854, 2)        uint8
x_test:   (27959, 500, 12)  float32
y_test:   (27959, 2)        uint8

 ------------------------------------------------------------------------


In [22]:
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(41469, 500, 12) (41469, 2)
(15854, 500, 12) (15854, 2)
(27959, 500, 12) (27959, 2)


In [26]:
model = cnn_model_1(x_train, y_train)
BATCH_SIZE = 32     # Typical values are 8, 16 or 32
NUM_EPOCHS = 30     # How many epochs to train the deep learning model

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print(model.summary())
start = time.time()
history = model.fit(
        x_train,y_train,
        steps_per_epoch=x_train.shape[0]//BATCH_SIZE,
        epochs=NUM_EPOCHS,
        validation_data=(x_valid,y_valid),
        validation_steps=x_valid.shape[0]//BATCH_SIZE)

print("Final Validation Accuracy: %0.3f" % history.history['val_accuracy'][-1])
end = time.time()
predictions = model.predict(x_test, verbose=0,batch_size=32)

#must use values not one-hot encoding, use argmax to convert
y_pred = np.argmax(predictions, axis=-1) # axis=-1 means last axis
y_test_act = np.argmax(y_test, axis=-1) # undo one-hot encoding

# Print print prediction accuracy
print('Prediction accuracy: {0:.3f}'.format(accuracy_score(y_test_act, y_pred)))
#acc_list_original.append(accuracy_score(y_test_act, y_pred))
print(end)
print(start)
print(end-start)
del model
del history

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 451, 100)       │        60,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 402, 100)       │       500,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 402, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 201, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 20100)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │     2,010,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           202 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,570,502 (9.81 MB)

 Trainable params: 2,570,502 (9.81 MB)

 Non-trainable params: 0 (0.00 B)

...

2024-04-26 19:33:00.965388: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1295/1295 ━━━━━━━━━━━━━━━━━━━━ 148s 115ms/step - accuracy: 0.7428 - loss: 0.5539 - val_accuracy: 0.7755 - val_loss: 0.5866
Epoch 3/30
   1/1295 ━━━━━━━━━━━━━━━━━━━━ 2:53 134ms/step - accuracy: 0.8182 - loss: 0.4898

2024-04-26 19:33:06.182928: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


 656/1295 ━━━━━━━━━━━━━━━━━━━━ 1:13 114ms/step - accuracy: 0.7577 - loss: 0.5386

KeyboardInterrupt: 

In [31]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Available GPUs:", gpus)
    
    # Set TensorFlow to only use the first GPU
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[5], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[5], True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)

Available GPUs: []
